In [855]:
import pandas as pd
import numpy as np
import re
import datetime

pd.options.display.max_rows = 999


In [819]:
ls

511_accts.csv                     florence.csv
Auxiliary_List.csv                florence_clean.csv
Cities_List.csv                   harvey.csv
Interstate_List.csv               harvey_clean.csv
List of 511 twitter accounts.csv  michael.csv
Road_Suffix_List.csv              michael_clean.csv
State_Road_List.csv               us_counties.csv
US_Routes.csv


In [820]:
harvey_df = pd.read_csv('harvey.csv')
michael_df = pd.read_csv('michael.csv')
florence_df = pd.read_csv('florence.csv')
florence_df = florence_df[florence_df.username != 'SouthDakotaDOT']

In [821]:
harvey_df['text'] = harvey_df['text'].map(lambda x: re.sub('http[s]?:\/\/[^\s]*',' ', x))
michael_df['text'] = michael_df['text'].map(lambda x: re.sub('http[s]?:\/\/[^\s]*',' ', x))
florence_df['text'] = florence_df['text'].map(lambda x: re.sub('http[s]?:\/\/[^\s]*',' ', x))


In [822]:
harvey_df['text'] = harvey_df['text'].str.lower()
michael_df['text'] = michael_df['text'].str.lower()
florence_df['text'] = florence_df['text'].str.lower()

In [823]:
# Converting column to datetime
harvey_df['date'] = pd.to_datetime(harvey_df['date'])
michael_df['date'] = pd.to_datetime(michael_df['date'])
florence_df['date'] = pd.to_datetime(florence_df['date'])
# Adding time column
harvey_df['time'] = harvey_df['date'].dt.time
michael_df['time'] = michael_df['date'].dt.time
florence_df['time'] = florence_df['date'].dt.time
# Making date only column
harvey_df['date'] = harvey_df['date'].dt.date
michael_df['date'] = michael_df['date'].dt.date
florence_df['date'] = florence_df['date'].dt.date

In [824]:
harvey_df.drop(columns=['hashtags','type','geo'], inplace=True)
michael_df.drop(columns=['geo','hashtags','type','geo'], inplace=True)
florence_df.drop(columns=['geo','hashtags','type','geo'], inplace=True)

In [850]:
harvey_df['text'] = harvey_df['text'].str.replace('.', '')
harvey_df['text'] = harvey_df['text'].str.replace('/', '')
harvey_df['text'] = harvey_df['text'].str.replace(',', '')
harvey_df['text'] = harvey_df['text'].str.replace("'", '')
harvey_df['text'] = harvey_df['text'].str.replace(";", '')
harvey_df['text'] = harvey_df['text'].str.replace("(", '')
harvey_df['text'] = harvey_df['text'].str.replace(")", '')


florence_df['text'] = florence_df['text'].str.replace('.', '')
florence_df['text'] = florence_df['text'].str.replace('/', '')
florence_df['text'] = florence_df['text'].str.replace(',', '')
florence_df['text'] = florence_df['text'].str.replace("'", '')
florence_df['text'] = florence_df['text'].str.replace(";", '')
florence_df['text'] = florence_df['text'].str.replace("(", '')
florence_df['text'] = florence_df['text'].str.replace(")", '')


michael_df['text'] = michael_df['text'].str.replace('.', '')
michael_df['text'] = michael_df['text'].str.replace('/', '')
michael_df['text'] = michael_df['text'].str.replace(',', '')
michael_df['text'] = michael_df['text'].str.replace("'", '')
michael_df['text'] = michael_df['text'].str.replace(";", '')
michael_df['text'] = michael_df['text'].str.replace("(", '')
michael_df['text'] = michael_df['text'].str.replace(")", '')

In [851]:
harvey_df.head(15)

,id,username,date,text,time,state,split_text
0,903345883609169921,TxDOTHouston,2017-08-31,turtle found refuge by entering the nearby wat...,19:56:56,Texas,"[turtle, found, refuge, by, entering, the, nea..."
1,903345706018197504,TxDOTHouston,2017-08-31,not a state road,19:56:14,Texas,"[not, a, state, road]"
2,903342514140217346,TxDOTHouston,2017-08-31,all ih 69 us 59 bridges over the brazos in for...,19:43:33,Texas,"[all, ih, 69, us, 59, bridges, over, the, braz..."
3,903340289095585792,TxDOTHouston,2017-08-31,of course,19:34:42,Texas,"[of, course]"
4,903334399692996609,TxDOTHouston,2017-08-31,houston district engineer quincy allen speaks ...,19:11:18,Texas,"[houston, district, engineer, quincy, allen, s..."
5,903333750439903232,TxDOTHouston,2017-08-31,sorry we are just following up with you this w...,19:08:43,Texas,"[sorry, we, are, just, following, up, with, yo..."
6,903331516268326912,TxDOTHouston,2017-08-31,us 59 ih 69 eastex nb at the san jacinto river...,18:59:51,Texas,"[us, 59, ih, 69, eastex, nb, at, the, san, jac..."
7,903321784191242240,TxDOTHouston,2017-08-31,keep going to to for updates,18:21:10,Texas,"[keep, going, to, to, for, updates]"
8,903321627022286848,TxDOTHouston,2017-08-31,no truth to rumor that ih 10 wb heading toward...,18:20:33,Texas,"[no, truth, to, rumor, that, ih, 10, wb, headi..."
9,903302735256268800,TxDOTHouston,2017-08-31,crews have begun resetting concrete barrier al...,17:05:29,Texas,"[crews, have, begun, resetting, concrete, barr..."


## Adding States

In [827]:
michael_df['state'] = michael_df['username'].map(lambda x: 'Georgia' if x in ['AlbanyGaPD','511Georgia','CityofAlbanyGA'] else 'Florida')



In [828]:
# harvey only has one state, TX
harvey_df['state'] = 'Texas'

In [829]:
florence_df['state'] = florence_df['username'].apply(lambda x: 'South Carolina' if x in ['SCDOTPeeDee' , 'SCDOTPiedmont','SCDOTMidlands','SCDOTLowCountry','SouthDakotaDOT'] else 'North Carolina')




In [831]:
# dictionary of each ID has two lists, seqeuntial one words, sequential two words 
# which wordz are in tweet - how words are positioned relative to eachother. 



In [832]:
florence_df['text']

0       driving in areas at risk of flooding is extrem...
1       bertie correctional institution assistant main...
2       find out more about fema disaster assistance a...
3                                                       …
4       if you evacuated during #hurricaneflorence it ...
                              ...                        
3441    collision i-26 wb: at exit154a no lns clsd ocr...
3442    cleared: vehicle on fire i-26 wb 1 mi w of exi...
3443    vehicle on fire i-26 wb: 1 mi w of exit177 all...
3444    cleared: traf congestion i-95 sb btwn 1 mi s o...
3445    update: traf congestion i-95 sb: btwn 1 mi s o...
Name: text, Length: 3446, dtype: object

## Split cell

In [833]:
harvey_df['split_text'] = harvey_df['text'].str.split()
michael_df['split_text'] = michael_df['text'].str.split()
florence_df['split_text'] = florence_df['text'].str.split()


In [834]:
florence_df['split_text'][6]

['great',
 'to',
 'see',
 'different',
 'teams',
 'from',
 'across',
 'the',
 'state',
 'coming',
 'together',
 'to',
 'provide',
 'relief',
 'to',
 'affected',
 'communities',
 '…',
 '#florencenc',
 '#rebuildnc',
 '#wearencdps']

In [835]:
florence_df['split_text'][1]

['bertie',
 'correctional',
 'institution',
 'assistant',
 'maintenance',
 'supervisor',
 'charles',
 'gurganus',
 'took',
 'action',
 'with',
 'several',
 'members',
 'of',
 'his',
 'church',
 'to',
 'help',
 'those',
 'affected',
 'by',
 'hurricane',
 'florence',
 'in',
 'martin',
 'county',
 '#wearencdps',
 '#florencencpictwittercommhbcmq6lui']

In [836]:
florence_df.to_csv('florence_clean.csv', index=False)
michael_df.to_csv('michael_clean.csv', index=False)
harvey_df.to_csv('harvey_clean.csv', index=False)

In [860]:
michael_df.head(9999)

,id,username,date,text,time,state,split_text
0,920328717519474689,NWSTallahassee,2017-10-17,tomorrow is fall severe weather awareness day ...,16:40:39,Florida,"[tomorrow, is, fall, severe, weather, awarenes..."
1,920328535608381442,NWSTallahassee,2017-10-17,tomorrow is fall severe weather awareness day ...,16:39:56,Florida,"[tomorrow, is, fall, severe, weather, awarenes..."
2,920250766115459072,NWSTallahassee,2017-10-17,the ashburn ga noaa weather radio will be down...,11:30:54,Florida,"[the, ashburn, ga, noaa, weather, radio, will,..."
3,920228486752677888,NWSTallahassee,2017-10-17,cooler weather has moved into our area: highs ...,10:02:22,Florida,"[cooler, weather, has, moved, into, our, area:..."
4,920106319029432322,NWSTallahassee,2017-10-17,another amazing sunset at panama city beach th...,01:56:55,Florida,"[another, amazing, sunset, at, panama, city, b..."
5,920050177062391809,NWSTallahassee,2017-10-16,it looks like the outage has been restored was...,22:13:50,Florida,"[it, looks, like, the, outage, has, been, rest..."
6,920030925144064001,NWSTallahassee,2017-10-16,thanks! do you know if that was due to wind or...,20:57:20,Florida,"[thanks!, do, you, know, if, that, was, due, t..."
7,920029814869970946,NWSTallahassee,2017-10-16,could you tell us if these were due to trees d...,20:52:55,Florida,"[could, you, tell, us, if, these, were, due, t..."
8,920021903338868739,NWSTallahassee,2017-10-16,severe thunderstorm warning including saint ma...,20:21:29,Florida,"[severe, thunderstorm, warning, including, sai..."
9,920012687886647305,NWSTallahassee,2017-10-16,it was a record warm first half of october in ...,19:44:52,Florida,"[it, was, a, record, warm, first, half, of, oc..."


## Making a list of incident

In [861]:
['Crash','Crashed','Accident','Collision','collided','Fatal','Tow',
'Break','Damage','damaged','Repair''Road work','Closure','closed',
'Zone','Maintenance','Schedule','Seal','Flooded','flood','broken','broke','Obstacles','Obstacle','Disabled',
'Stuck','Rain','Slip','Wind','Flood','Rainy','Hazard','Tree','Block','Wiper','Inches','Wet',
'Cold','Freeze','Hot','Visibility','Fire','Weather','Animal','Deer','Dead','Hail','Melt','Slope',
'Slick','Tire','Cover','Friction','Frozen', 'storm', 'traffic']

['Crash',
 'Crashed',
 'Accident',
 'Collision',
 'collided',
 'Fatal',
 'Tow',
 'Break',
 'Damage',
 'damaged',
 'RepairRoad work',
 'Closure',
 'closed',
 'Zone',
 'Maintenance',
 'Schedule',
 'Seal',
 'Flooded',
 'flood',
 'broken',
 'broke',
 'Obstacles',
 'Obstacle',
 'Disabled',
 'Stuck']